In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, precision_recall_curve, ConfusionMatrixDisplay, roc_curve, auc, roc_auc_score
from itertools import cycle 

In [36]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [38]:
# Load datasets
data_dir = "D:/MLUP Proejct/Alzheimer_s Dataset"
train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=data_transforms['test'])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
# Load pre-trained ResNet18 and modify the final layer
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4 classes
model = model.to(device)

In [40]:
# Loss, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

In [ ]:
# Increase epochs to 50
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=50):
    model.train()
    best_accuracy = 0.0
    train_loss = []
    train_acc = []
    train_precision = []

    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0.0
        correct = 0
        total = 0
        true_positives = 0
        predicted_positives = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            true_positives += ((predicted == labels) & (predicted == 1)).sum().item()
            predicted_positives += (predicted == 1).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        epoch_precision = 100 * true_positives / predicted_positives if predicted_positives > 0 else 0
        
        scheduler.step(epoch_loss)
        
        train_loss.append(epoch_loss)
        train_acc.append(epoch_acc)
        train_precision.append(epoch_precision)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%, Precision: {epoch_precision:.2f}%, Time: {time.time()-start_time:.2f}s")
        
        if epoch_acc > best_accuracy:
            best_accuracy = epoch_acc
            torch.save(model.state_dict(), 'best_model.pth')

    torch.save(model.state_dict(), 'final_model.pth')
    print("Training Complete. Best Accuracy: {:.2f}%".format(best_accuracy))
    plot_training_curves(train_loss, train_acc, train_precision)

In [42]:
# Evaluate model with precision and confusion matrix
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    true_positives = 0
    predicted_positives = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            true_positives += ((predicted == labels) & (predicted == 1)).sum().item()
            predicted_positives += (predicted == 1).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    precision = 100 * true_positives / predicted_positives if predicted_positives > 0 else 0
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Test Precision: {precision:.2f}%")
    
    # Plot confusion matrix
    plot_confusion_matrix(all_labels, all_preds)
    # Plot precision-recall curve
    plot_precision_recall_curve(all_labels, all_preds)

In [43]:
# Plot training loss, accuracy, and precision
def plot_training_curves(train_loss, train_acc, train_precision):
    epochs = range(1, len(train_loss) + 1)
    plt.figure(figsize=(15, 5))

    # Loss Plot
    plt.subplot(1, 3, 1)
    plt.plot(epochs, train_loss, 'r-', label="Training Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training Loss vs. Epochs")
    plt.legend()

    # Accuracy Plot
    plt.subplot(1, 3, 2)
    plt.plot(epochs, train_acc, 'b-', label="Training Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.title("Training Accuracy vs. Epochs")
    plt.legend()

    # Precision Plot
    plt.subplot(1, 3, 3)
    plt.plot(epochs, train_precision, 'g-', label="Training Precision")
    plt.xlabel("Epochs")
    plt.ylabel("Precision (%)")
    plt.title("Training Precision vs. Epochs")
    plt.legend()

    plt.tight_layout()
    plt.show()

In [44]:
# Plot confusion matrix
def plot_confusion_matrix(labels, preds):
    cm = confusion_matrix(labels, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Mild", "Moderate", "Non", "VeryMild"])
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()

In [45]:
# Plot precision-recall curve (simplified for multiclass)
def plot_precision_recall_curve(labels, preds):
    # Convert to binary for simplicity (e.g., class 1 vs rest)
    labels_bin = [1 if x == 1 else 0 for x in labels]
    preds_bin = [1 if x == 1 else 0 for x in preds]
    precision, recall, _ = precision_recall_curve(labels_bin, preds_bin)
    
    plt.figure(figsize=(6, 6))
    plt.plot(recall, precision, marker='.')
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve (Class 1 vs Rest)")
    plt.grid()
    plt.show()

In [ ]:
# Plot ROC-AUC curve for multiclass classification
def plot_roc_auc_curve(labels, preds, num_classes=4):
    labels_one_hot = np.eye(num_classes)[labels]
    preds_one_hot = np.eye(num_classes)[preds]
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(labels_one_hot[:, i], preds_one_hot[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    plt.figure(figsize=(8, 8))
    colors = cycle(['blue', 'red', 'green', 'purple'])
    for i, color in zip(range(num_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'Class {i} (AUC = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC-AUC Curve')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

# Evaluate model with ROC-AUC
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    true_positives = 0
    predicted_positives = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            true_positives += ((predicted == labels) & (predicted == 1)).sum().item()
            predicted_positives += (predicted == 1).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    precision = 100 * true_positives / predicted_positives if predicted_positives > 0 else 0
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Test Precision: {precision:.2f}%")
    
    # Plot confusion matrix
    plot_confusion_matrix(all_labels, all_preds)
    # Plot precision-recall curve
    plot_precision_recall_curve(all_labels, all_preds)
    # Plot ROC-AUC curve
    plot_roc_auc_curve(all_labels, all_preds)


In [ ]:
# Train and evaluate
train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=50)
evaluate_model(model, test_loader)